In [1]:
import matplotlib.pyplot as plt

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision.datasets as datasets


assert torch.cuda.is_available(), "GPU is not enabled"

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Declare the transform
transform = transforms.Compose(
    [transforms.ToTensor(), # Transform image to tensor
     transforms.Normalize((0.5,),(0.5,))] # Normalize data
)

# Declare training and testing sets
train_set = datasets.FashionMNIST('data',
  download=True,
  train=True,
  transform=transform
)
test_set = datasets.FashionMNIST('data',
  download=True,
  train=False,
  transform=transform
)

# Declare dataloaders
train_loader = 

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')



device(type='cuda', index=0)